<a href="https://colab.research.google.com/github/fareed37/Capstone-Projects/blob/main/Capstone_2_Employee_attrition_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mysql.connector

Processing /root/.cache/pip/wheels/8c/83/a1/f8b6d4bb1bd6208bbde1608bbfa7557504bed9eaf2ecf8c175/mysql_connector-2.2.9-cp36-cp36m-linux_x86_64.whl


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Reading and Exploring Data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,auc,roc_auc_score
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
  host="cpanel.insaid.co",
  user="student",
  password="student",
  database="Capstone2"
)

tables = ['department_data', 'employee_details_data', 'employee_data']

#mycursor = mydb.cursor()
mycursor = mydb.cursor(buffered=True , dictionary=True)
mycursor.execute("SELECT * FROM department_data")
department_data = pd.DataFrame(mycursor.fetchall())

mycursor.execute("SELECT * FROM employee_data")
employee_data = pd.DataFrame(mycursor.fetchall())

mycursor.execute("SELECT * FROM employee_details_data")
employee_details_data = pd.DataFrame(mycursor.fetchall())



#for x in department_data:
print(len(department_data))
print(len(employee_data))
print(len(employee_details_data))


11
14150
14245


### Exploring department_data

In [ ]:
department_data.head(15)

,dept_id,dept_name,dept_head
0,D00-IT,IT,Henry Adey
1,D00-SS,Sales,Edward J Bayley
2,D00-TP,Temp,Micheal Zachrey
3,D00-ENG,Engineering,Sushant Raghunathan K
4,D00-SP,Support,Amelia Westray
5,D00-FN,Finance,Aanchal J
6,D00-PR,Procurement,Louie Viles
7,D00-AD,Admin,Evelyn Tolson
8,D00-MN,Management,Ellie Trafton
9,D00-MT,Marketing,Reuben Swann


In [ ]:
department_data.describe()

,dept_id,dept_name,dept_head
count,11,11,11
unique,11,11,11
top,D00-PD,Finance,Ellie Trafton
freq,1,1,1


In [ ]:
department_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   dept_id    11 non-null     object
 1   dept_name  11 non-null     object
 2   dept_head  11 non-null     object
dtypes: object(3)
memory usage: 392.0+ bytes


### Exploring employee_data

In [ ]:
employee_data.head(10)

,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id
0,246.00,None,None,0.866838,6,NaN,medium,0.134415,Left,4.00,124467
1,134.00,None,None,0.555718,2,NaN,low,0.511041,Left,3.00,112210
2,156.00,D00-SS,1.00,0.474082,2,NaN,medium,0.405101,Left,3.00,126150
3,256.00,D00-SP,None,0.961360,6,NaN,low,0.152974,Left,4.00,125346
4,146.00,D00-SS,None,0.507349,2,NaN,medium,0.434845,Left,3.00,113707
5,135.00,D00-MT,None,0.482184,2,NaN,low,0.381545,Left,3.00,121685
6,270.00,D00-PD,None,0.867087,6,NaN,low,0.172575,Left,4.00,119912
7,244.00,D00-IT,None,0.926197,6,NaN,medium,0.061868,Left,5.00,112868
8,289.00,D00-IT,None,0.929858,7,NaN,low,0.161744,Left,4.00,122314
9,281.00,D00-SS,None,0.907965,6,NaN,medium,0.105749,Left,4.00,122394


In [ ]:
employee_data.describe()

,last_evaluation,n_projects,recently_promoted,satisfaction,employee_id
count,12663.000000,14150.000000,297.0,14000.000000,14150.000000
mean,0.718399,3.778304,1.0,0.621212,112080.750247
std,0.173108,1.250162,0.0,0.250482,8748.202856
min,0.316175,1.000000,1.0,0.040058,0.000000
25%,0.563711,3.000000,1.0,0.450356,105772.500000
50%,0.724731,4.000000,1.0,0.652394,111291.500000
75%,0.871409,5.000000,1.0,0.824925,116650.750000
max,1.000000,7.000000,1.0,1.000000,148988.000000


In [ ]:
employee_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14150 entries, 0 to 14149
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   avg_monthly_hrs    14150 non-null  object 
 1   department         13443 non-null  object 
 2   filed_complaint    2046 non-null   object 
 3   last_evaluation    12663 non-null  float64
 4   n_projects         14150 non-null  int64  
 5   recently_promoted  297 non-null    float64
 6   salary             14150 non-null  object 
 7   satisfaction       14000 non-null  float64
 8   status             14150 non-null  object 
 9   tenure             14000 non-null  object 
 10  employee_id        14150 non-null  int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 1.2+ MB


**Note** : employee_data table has missing data in the below columns:<br>
a. tenure (150 records) <br>
b. satisfaction(150 records) <br>
c. last_evaluation(1487 records) <br>
d. department(707 records) <br>
e. filed_commplaint(12104) <br>
f. recently_promoted(13853)<br>

Missing data has to be handled.

In [ ]:
employee_data.employee_id.nunique()

14117

**Note**: employee_data has only 14117 unique employee ids our of 14150 records. Looks like there are 33 duplicate records

In [ ]:
employee_data.employee_id.duplicated().sum()

33

In [ ]:
employee_data[employee_data.employee_id.duplicated(keep=False)]

,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id
34,284.00,D00-SS,None,0.852702,6,NaN,low,0.069168,Left,4.00,0
304,264.00,D00-ENG,None,0.774554,6,NaN,low,0.110974,Left,4.00,0
374,307.00,D00-IT,None,0.905708,7,NaN,low,0.125275,Left,4.00,111323
1234,129.00,D00-SP,None,0.402660,2,NaN,medium,0.437359,Left,3.00,0
1438,282.00,D00-SS,None,0.875427,6,NaN,low,0.065908,Left,4.00,116247
...,...,...,...,...,...,...,...,...,...,...,...
14145,245.00,D00-SS,None,0.850785,6,NaN,medium,0.225084,Employed,5.00,101809
14146,192.00,D00-SS,None,0.951901,3,NaN,medium,0.539834,Employed,3.00,116153
14147,175.00,D00-MN,None,0.854538,4,NaN,medium,0.934465,Employed,3.00,119780
14148,268.00,D00-MT,None,0.900887,5,NaN,medium,0.906613,Left,5.00,115325


**Note** : There are 33 repeated employee ids. Also, we noted that there are 4 employee_ids with value 0. Duplicate records needs to be dropped and employee_ids with value 0 has to be deleted.

In [ ]:
employee_data.nunique()

avg_monthly_hrs        249
department              12
filed_complaint          1
last_evaluation      12188
n_projects               7
recently_promoted        1
salary                   3
satisfaction         13497
status                   2
tenure                   8
employee_id          14117
dtype: int64

In [ ]:
employee_data.department.unique()

array([None, 'D00-SS', 'D00-SP', 'D00-MT', 'D00-PD', 'D00-IT', 'D00-AD',
       'D00-MN', 'D00-ENG', 'D00-PR', 'D00-TP', 'D00-FN', '-IT'],
      dtype=object)

**Note**: There are 12 departments in employee_data where as in department_data there are only 11 departments. emplyee_data has an additional department: '-IT'. Looks like this is an error while entering the data. The department should have been 'D00-IT'

In [ ]:
employee_data[employee_data['employee_id'] == 0]

,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id
34,284.00,D00-SS,None,0.852702,6,NaN,low,0.069168,Left,4.00,0
304,264.00,D00-ENG,None,0.774554,6,NaN,low,0.110974,Left,4.00,0
1234,129.00,D00-SP,None,0.402660,2,NaN,medium,0.437359,Left,3.00,0
11304,167.00,None,None,0.880053,5,NaN,low,0.682140,Employed,3.00,0
12304,259.00,D00-ENG,None,0.505883,5,NaN,low,0.631046,Employed,4.00,0


In [ ]:
employee_data.status.value_counts()

Employed    10784
Left         3366
Name: status, dtype: int64

#### Exploring employee_details_data

In [ ]:
employee_details_data.head()

,employee_id,age,gender,marital_status
0,113558,43,Male,Married
1,112256,24,Female,Unmarried
2,112586,22,Female,Unmarried
3,108071,36,Male,Married
4,116915,38,Male,Married


In [ ]:
employee_details_data.describe()

,employee_id,age
count,14245.000000,14245.000000
mean,112123.050544,32.889926
std,8500.457343,9.970834
min,100101.000000,22.000000
25%,105775.000000,24.000000
50%,111298.000000,29.000000
75%,116658.000000,41.000000
max,148988.000000,57.000000


In [ ]:
employee_details_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14245 entries, 0 to 14244
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   employee_id     14245 non-null  int64 
 1   age             14245 non-null  int64 
 2   gender          14245 non-null  object
 3   marital_status  14245 non-null  object
dtypes: int64(2), object(2)
memory usage: 445.3+ KB


In [ ]:
employee_details_data.employee_id.nunique()

14245

**Note**: There are 14245 unique employee_ids in employee_details_data. Where as in employee_data, there are only 14117 unique records. Employee_data has 128 employee_ids missing. 

In [ ]:
employee_details_data[employee_details_data['employee_id'] == 0]

,employee_id,age,gender,marital_status


# 	2. Fix Structural Errors

### Dropping duplicate columns:

In [ ]:
employee_data = employee_data.drop_duplicates()

### Dropping employee_id=0 columns:

In [ ]:
employee_data[employee_data['employee_id'] == 0].index

Int64Index([34, 304, 1234, 11304, 12304], dtype='int64')

In [ ]:
employee_data = employee_data.drop(employee_data[employee_data['employee_id'] == 0].index)
#data.drop(["EventID"], axis = 1, inplace = True)

In [ ]:
employee_data.count()

avg_monthly_hrs      14116
department           13410
filed_complaint       2041
last_evaluation      12629
n_projects           14116
recently_promoted      297
salary               14116
satisfaction         13966
status               14116
tenure               13966
employee_id          14116
dtype: int64

In [ ]:
employee_data.head(10)

,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id
0,246.00,None,None,0.866838,6,NaN,medium,0.134415,Left,4.00,124467
1,134.00,None,None,0.555718,2,NaN,low,0.511041,Left,3.00,112210
2,156.00,D00-SS,1.00,0.474082,2,NaN,medium,0.405101,Left,3.00,126150
3,256.00,D00-SP,None,0.961360,6,NaN,low,0.152974,Left,4.00,125346
4,146.00,D00-SS,None,0.507349,2,NaN,medium,0.434845,Left,3.00,113707
5,135.00,D00-MT,None,0.482184,2,NaN,low,0.381545,Left,3.00,121685
6,270.00,D00-PD,None,0.867087,6,NaN,low,0.172575,Left,4.00,119912
7,244.00,D00-IT,None,0.926197,6,NaN,medium,0.061868,Left,5.00,112868
8,289.00,D00-IT,None,0.929858,7,NaN,low,0.161744,Left,4.00,122314
9,281.00,D00-SS,None,0.907965,6,NaN,medium,0.105749,Left,4.00,122394


In [ ]:
employee_data['filed_complaint'] = employee_data['filed_complaint'].fillna(0)
employee_data['recently_promoted'] = employee_data['recently_promoted'].fillna(0)
employee_data['last_evaluation'] = employee_data['last_evaluation'].fillna(0.0)
employee_data['satisfaction'] = employee_data['satisfaction'].fillna(0.0)
employee_data['tenure'] = employee_data['tenure'].fillna(0.0)

### Replace wrong department value '-IT' with 'D00-IT'

In [ ]:

#employee_data['department'] = employee_data.apply(lambda x: 'D00-IT' if(employee_data['department'] == '-IT' or )  else employee_data['department'], axis =1)

employee_data['department'] = employee_data['department'].replace({'-IT': 'D00-IT'})


In [ ]:
employee_data.department.unique()

array([None, 'D00-SS', 'D00-SP', 'D00-MT', 'D00-PD', 'D00-IT', 'D00-AD',
       'D00-MN', 'D00-ENG', 'D00-PR', 'D00-TP', 'D00-FN'], dtype=object)

#### Replacing missing department with mode value

In [ ]:
employee_data['department'] = employee_data['department'].fillna(employee_data['department'].mode()[0])

In [ ]:
employee_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14116 entries, 0 to 14120
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   avg_monthly_hrs    14116 non-null  object 
 1   department         14116 non-null  object 
 2   filed_complaint    14116 non-null  object 
 3   last_evaluation    14116 non-null  float64
 4   n_projects         14116 non-null  int64  
 5   recently_promoted  14116 non-null  float64
 6   salary             14116 non-null  object 
 7   satisfaction       14116 non-null  float64
 8   status             14116 non-null  object 
 9   tenure             14116 non-null  object 
 10  employee_id        14116 non-null  int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 1.3+ MB


**Note**: avg_monthly_hrs, filed_complaint, salary,tenure are considered object types where as they have float values iin those fields. So changing the type of the column to float

In [ ]:
employee_data['avg_monthly_hrs'] =employee_data['avg_monthly_hrs'].astype('float')
employee_data['filed_complaint'] =employee_data['filed_complaint'].astype('int')
employee_data['tenure'] =employee_data['tenure'].astype('float')

In [ ]:
employee_data.filed_complaint.value_counts()

0    12075
1     2041
Name: filed_complaint, dtype: int64

# Merging Data

In [ ]:
train_dataset = pd.merge(employee_details_data, employee_data, on='employee_id', how='right') #.merge(department_data, on='department', how='left')

In [ ]:
train_dataset.head()

,employee_id,age,gender,marital_status,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure
0,124467,42,Female,Married,246.0,D00-SS,0,0.866838,6,0.0,medium,0.134415,Left,4.0
1,112210,23,Female,Unmarried,134.0,D00-SS,0,0.555718,2,0.0,low,0.511041,Left,3.0
2,126150,24,Female,Unmarried,156.0,D00-SS,1,0.474082,2,0.0,medium,0.405101,Left,3.0
3,125346,51,Female,Married,256.0,D00-SP,0,0.961360,6,0.0,low,0.152974,Left,4.0
4,113707,23,Female,Unmarried,146.0,D00-SS,0,0.507349,2,0.0,medium,0.434845,Left,3.0


In [ ]:
train_dataset.to_excel("employee_data.xlsx") 

In [ ]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14116 entries, 0 to 14115
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   employee_id        14116 non-null  int64  
 1   age                14116 non-null  int64  
 2   gender             14116 non-null  object 
 3   marital_status     14116 non-null  object 
 4   avg_monthly_hrs    14116 non-null  float64
 5   department         14116 non-null  object 
 6   filed_complaint    14116 non-null  int64  
 7   last_evaluation    14116 non-null  float64
 8   n_projects         14116 non-null  int64  
 9   recently_promoted  14116 non-null  float64
 10  salary             14116 non-null  object 
 11  satisfaction       14116 non-null  float64
 12  status             14116 non-null  object 
 13  tenure             14116 non-null  float64
dtypes: float64(5), int64(4), object(5)
memory usage: 1.6+ MB


In [ ]:
train_dataset.describe()

,employee_id,age,avg_monthly_hrs,filed_complaint,last_evaluation,n_projects,recently_promoted,satisfaction,tenure
count,14116.000000,14116.000000,14116.000000,14116.000000,14116.000000,14116.000000,14116.000000,14116.000000,14116.000000
mean,112120.657764,32.896005,199.992632,0.144588,0.642653,3.777770,0.021040,0.614719,3.460541
std,8497.639403,9.975000,50.826952,0.351697,0.274645,1.249693,0.143523,0.257143,1.496271
min,100101.000000,22.000000,49.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,105773.500000,24.000000,155.000000,0.000000,0.522018,3.000000,0.000000,0.442808,3.000000
50%,111293.500000,29.000000,199.000000,0.000000,0.683402,4.000000,0.000000,0.648350,3.000000
75%,116655.250000,41.000000,245.000000,0.000000,0.856854,5.000000,0.000000,0.822960,4.000000
max,148988.000000,57.000000,310.000000,1.000000,1.000000,7.000000,1.000000,1.000000,10.000000


In [ ]:
train_dataset.columns

Index(['employee_id', 'age', 'gender', 'marital_status', 'avg_monthly_hrs',
       'department', 'filed_complaint', 'last_evaluation', 'n_projects',
       'recently_promoted', 'salary', 'satisfaction', 'status', 'tenure'],
      dtype='object')

In [ ]:
train_dataset.marital_status.value_counts()

Unmarried    7211
Married      6905
Name: marital_status, dtype: int64

In [ ]:
gg["filed_complaint"].value_counts()

NameError: ignored

# 3. EDA

#### Distribution of Left and Employed numbers

In [ ]:
train_dataset['status'].value_counts().plot(kind='pie',  explode = [0, 0.09], fontsize=14, autopct='%3.1f%%', 
                                               figsize=(7,7), shadow=True, startangle=135, legend=True, cmap='summer')
plt.title('Pie chart showing the proportion of employees who have left the organisation')

**Observations** : 23.8% of the employees have left and 76.2% of the employees are still employed with the company

In [ ]:
cont = ['satisfaction', 'avg_monthly_hrs', 'n_projects', 'tenure', 'age', 'last_evaluation']
cats = list(set(train_dataset.columns)-set(cont))
cats.remove('employee_id')
cats.remove('status')
print(cats)
fig, axes = plt.subplots(3, 2, figsize=(15, 15))
axes = [ax for axes_row in axes for ax in axes_row]

for i, c in enumerate(train_dataset[cats]):
    _ = train_dataset[c].value_counts()[::-1].plot(kind = 'pie', ax=axes[i], shadow = True, title=c, autopct='%.0f',cmap = "Spectral", fontsize=12, startangle = 30,wedgeprops=dict(width=0.15))
    _ = axes[i].set_ylabel('')
    
_ = plt.tight_layout()


In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
sns.heatmap( train_dataset[['age', 'avg_monthly_hrs', 'filed_complaint', 'last_evaluation','n_projects','recently_promoted','satisfaction','tenure'] ].corr(), annot=True );

#### How are married people who are working long hours coping in the company

In [ ]:
sns.factorplot(y ='marital_status', x= 'avg_monthly_hrs',kind = 'violin', col = 'gender', 
               hue= 'status',data = train_dataset, palette= 'RdBu', split = True)

**Observations**: Interestingly, most of the married people who have left the company have been working longer hours compared to married people who are still employed. Contrary to married people, Unmarried people who have left the company are interestingly working less than unmarried people who have stayed with the company. 

#### Recently promoted vs average monthly working hours

In [ ]:
sns.factorplot(x ='recently_promoted', y= 'avg_monthly_hrs',kind = 'violin', col='gender',
               hue= 'status',data = train_dataset, palette='Set3', split = True)

** Observations** : When we see how many hours an employees who is promoted working, we notice that a promoted employee if he has intention to leave, reduces his working hours.

#### Recently promoted vs average monthly working hours

In [ ]:
g = sns.catplot(x ='marital_status', 
               hue= 'status',kind = 'count', col = 'gender', data = train_dataset, palette="Set1")
total = len(train_dataset['marital_status'])
for ax in g.axes.ravel():
  for p in ax.patches:
    ax.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

**Observations**: for around 3.7k male employees (either married or not), 900+ employees are quitting. That is around 25% of employees. when we consider Female employees, for 1726 married female employees 471 have left which makes 27%. where as 1024 unmarried female employees our of 1608, ie., 63% of unmarried female employees have left the company. 

# 3. Preprocessing

In [ ]:
train_dataset = pd.get_dummies(train_dataset, columns=["gender","marital_status","status","salary","department"], drop_first=True)

#### dropping employee_id column

In [ ]:
dropColumns = ['employee_id']
train_dataset = train_dataset.drop(dropColumns,axis=1)

In [ ]:
train_dataset.head()

In [ ]:
train_dataset.columns

In [ ]:
train_dataset= train_dataset.rename(columns={"status_Left": "status"}, errors="raise")

### Finding the importance of the parameters

In [ ]:
status = train_dataset.pop('status')


In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(status,train_dataset)
result=logit_model.fit()
print(result.summary2())


**Observations**: satisfaction is the most effective parameter. Followed by filed_complaints, gender and salary

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

train_dataset.pop('age')
vif_data= pd.DataFrame()


vif_data['feature'] = train_dataset.columns
vif_data["VIF"]  = [variance_inflation_factor(train_dataset.values, i) for i in range(len(train_dataset.columns))]
print(vif_data)

### Using StandardScaler to standardize the data

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

sc.fit(train_dataset)                                                                                   # fit the data to Standardscaler
train_dataset_scale = sc.transform(train_dataset)                                                               # transform the data

print('After standardizing our features, the first 5 rows of our data now look like this:\n')   # print the first 5 value after standardization
print(pd.DataFrame(train_dataset_scale, columns=train_dataset.columns).head())                                  # Put it to dataframe                              
print(type(train_dataset_scale))

## Train-Test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_dataset_scale, status, test_size = 0.3, random_state = 42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## 6. User defined functions:

#### Defining user defined functions to reuse them with each algorithm

<a id=section601></a>
#### 6.1 User defined function to fit and predict the model

In [ ]:
#Function for prediction

def get_predictions(clf, X_train, y_train, X_test, y_test):
    # fit it to training data
    clf.fit(X_train,y_train)
    # predict using test data
    y_pred = clf.predict(X_test)
    # Compute predicted probabilities: y_pred_prob
    y_pred_prob = clf.predict_proba(X_test)
    
   # print('train-set confusion matrix:\n', confusion_matrix(y_test, y_pred)) 
    return y_pred, y_pred_prob

<a id=section602></a>
#### 6.2 User defined function to print the classifier score

The function is returning accuracy_score as the goal of the analysis is just to predict the voice type and we are not interested in increasing precision or recall here. Also, since data is equally distributed, accuracy score works best and it is sufficient for us in the analysis. Though the function prints confusion matrix, recall score, precision score, f1 score, ROC AUC and also accuracy score, we are interested only in accuracy score for the project

In [ ]:
def print_scores(y_test,y_pred,y_pred_prob):
    print('test-set confusion matrix:\n', confusion_matrix(y_test,y_pred)) 
    print("recall score: ", recall_score(y_test,y_pred))
    print("precision score: ", precision_score(y_test,y_pred))
    print("f1 score: ", f1_score(y_test,y_pred))
    print("accuracy score: ", accuracy_score(y_test,y_pred))
    print("ROC AUC: {}".format(roc_auc_score(y_test, y_pred_prob[:,1])))
    return  accuracy_score(y_test,y_pred)

<a id=section7></a>
## 7. Modelling and prediction

Applying various models to see which one gives a better result. We are trying:
1. Gaussian Naive Bayees
2. K-Nearest Classifier
3. Logistic regression
4. LDA

<a id=section701></a>
## 7.1 Gaussian Naive Bayes

A naive Bayes classifier uses probability theory to classify data. Naive Bayes classifier algorithms make use of Bayes' theorem. The key insight of Bayes' theorem is that the probability of an event can be adjusted as new data is introduced.

In [ ]:
y_pred_nb, y_pred_prob_nb = get_predictions(GaussianNB(), X_train, y_train, X_test, y_test)

In [ ]:
print_scores(y_test,y_pred_nb,y_pred_prob_nb)

<a id=section702></a>
## 7.2 KNearest Neighbour Classifier

K-Nearest Neighbours is one of the most basic yet essential classification algorithms which belongs to the supervised learning domain. K-Nearest neighbour can be used as both regressor and for classification.

When KNN is used for classification, the output can be calculated as the class with the highest frequency from the K-most similar instances. Each instance in essence votes for their class and the class with the most votes is taken as the prediction.

it is a good idea to choose a K value with an odd number to avoid a tie. And the inverse, use an even number for K when you have an odd number of classes.

KNN Algorithm is based on feature similarity: Choosing the right value of k is a process called parameter tuning, and is important for better accuracy. 

So in the below code We are checking the kNN for various value of K. K values range from 1 to 28 (28 being sqrt(792). Here 792 is the number of test data points.

### **7.2.1 Hyperparameter Tuning: Finding Optimal K**

- We **will** **iterate** our model **over** **some** **iterations**, finding optimal K value for clustering.

- We **check** **inertia**, defined as the mean squared distance between each instance and its closest centroid. Logically, as per the definition **lower** the **inertia** **better** the **model**.

- We will **use** **Elbow** **rule** in order to **find** the **optimal** **number** of **clusters**.

In [ ]:
status.value_counts()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
k_list = list(range(1,15,2))
# creating list of cv scores
cv_scores = []

# perform 10-fold cross validation
for k in k_list:
    print("K=",k)
    knn = KNeighborsClassifier(n_neighbors=k)
    y_pred_knn, y_pred_prob_knn = get_predictions(knn, X_train, y_train, X_test, y_test)
    scores = print_scores(y_test,y_pred_knn,y_pred_prob_knn)
    cv_scores.append(scores)
print(cv_scores)
#knn = KNeighborsClassifier(n_neighbors=7)
#y_pred_knn, y_pred_prob_knn = get_predictions(knn, X_train, y_train, X_test, y_test)

The accuracy_scores of different k values can be plotted to see which k value to chose

In [ ]:
plt.figure()
plt.figure(figsize=(15,10))
plt.title('The optimal number of neighbors', fontsize=20, fontweight='bold')
plt.xlabel('Number of Neighbors K', fontsize=15)
plt.ylabel('Accuracy score', fontsize=15)
sns.set_style("whitegrid")
plt.plot(k_list, cv_scores)

plt.show()

**Observations**: Accuracy score is highest for k=3. We will use that value for our model

<a id=section703></a>
## 7.3 Using Logistic Regression

Logistic regression is a supervised learning techinques which helps in classification problems.Logistic Regression is used when the dependent variable(target) is categorical. Sigmoid function or logistic function is used as hypothesis function for logistic regression.

In [ ]:
# let us check recall score for logistic regression
y_pred_lr, y_pred_prob_lr = get_predictions(LogisticRegression(C = 0.01, penalty = 'l1',solver='liblinear')
                                      , X_train, y_train, X_test, y_test)

In [ ]:
print_scores(y_test,y_pred_lr,y_pred_prob_lr)

<a id=section704></a>
## 7.4 Using LDA:

Linear Discriminant Analysis is a __dimensionality reduction__ technique used as a __preprocessing step__ in Machine Learning and __pattern classification applications__.

Linear Discriminant Analysis is a __supervised classification technique__ which takes labels into consideration.

In [ ]:
# let us check recall score for logistic regression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components = 1)
y_pred_lda, y_pred_prob_lda = get_predictions(lda, X_train, y_train, X_test, y_test)

In [ ]:
print_scores(y_test,y_pred_lda,y_pred_prob_lda)

In [ ]:
<a id=section704></a>
## 7.4 Create Boosting Classifiers

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
reg_ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1))
scores_ada = cross_val_score(reg_ada, X_train, y_train, cv=6)
scores_ada.mean()

In [ ]:
score = []
for depth in [1,2,4,6,8,10] : 
    reg_ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=depth))
    scores_ada = cross_val_score(reg_ada, X_train, y_train, cv=6)
    score.append(scores_ada.mean())


print(score)

In [ ]:
**Observations** : And the maximal score is reached for a depth of 10 in this simple example, with an accuracy of 97.45%

In [ ]:
<a id="section7"></a>
## 7. Model Comparison

In [ ]:
| Algorithms                                                                                     | Accuracy Score                                                                               |
| ------------------------------- |:-----------------------------------------------------------------------------------------:| 
| __Logistic Regression__ | __58.5%__ |
| __Random Forest__ | __64.0%__ |
| __Decision Tree Classifier__| __60.2%__| 
| __Bagging Classifier__ | __64.6%__ |
| __Boosting Classifier__ | __62.8%__ |
| __SVC__| __51.3%__| 
| __Voting Classifier__| __65.1%__| 